# Generic Time-Series Anomaly Detection using Autoencoder

Embedded & TinyML Friendly Student Template

## Objective
This notebook demonstrates a generic anomaly detection approach for time-series sensor data using an autoencoder.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Configuration (Edit Only This Section)

•	Tensors: Models operate using tensors, essentially lists or arrays of numbers. A 1D tensor is a vector, a 2D tensor is a matrix, and a single number is a scalar.
•	Feature Generation: Raw data, often collected as time series with inconsistent intervals, must be prepared for the model. A common technique is windowing, where a chunk of time is defined, and values within that window are combined (e.g., by averaging) to create a single set of input features.
•	Normalization: For the training algorithm to work effectively, feature values must be scaled, ideally into a similar range (like 0 to 1). For example, 8-bit image data (0-255) is normalized by multiplying values by 1/255.

In [ ]:

WINDOW_SIZE = 26
NUM_FEATURES = 1
NORMALIZATION_SCALE = 2048
THRESHOLD_PERCENTILE = 95


## Load Sensor Data

In [ ]:

def load_sensor_data(file_path):
    return np.loadtxt(file_path, delimiter=',')

raw_data = load_sensor_data("normal_data.csv")
print(raw_data.shape)


## Sliding Window Creation

In [ ]:

def create_windows(data, window_size):
    windows = []
    for i in range(len(data) - window_size):
        windows.append(data[i:i+window_size])
    return np.array(windows)

X = create_windows(raw_data, WINDOW_SIZE)
print(X.shape)


## Normalization

In [ ]:
X = X / NORMALIZATION_SCALE

## Autoencoder Model

In [ ]:

autoencoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(WINDOW_SIZE, NUM_FEATURES)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(WINDOW_SIZE * NUM_FEATURES),
    tf.keras.layers.Reshape((WINDOW_SIZE, NUM_FEATURES))
])

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()


## Training

In [ ]:

history = autoencoder.fit(X, X, epochs=30, batch_size=32, validation_split=0.1)


## Reconstruction Error

In [ ]:

X_recon = autoencoder.predict(X)
recon_error = np.mean(np.square(X - X_recon), axis=(1,2))

plt.hist(recon_error, bins=50)
plt.title("Reconstruction Error (Normal Data)")
plt.show()


## Threshold Selection

In [ ]:

THRESHOLD = np.percentile(recon_error, THRESHOLD_PERCENTILE)
print("Threshold:", THRESHOLD)


## Testing on New Data

In [ ]:

test_data = load_sensor_data("test_data.csv")
test_windows = create_windows(test_data, WINDOW_SIZE)
test_windows = test_windows / NORMALIZATION_SCALE

test_recon = autoencoder.predict(test_windows)
test_error = np.mean(np.square(test_windows - test_recon), axis=(1,2))

plt.plot(test_error, label='Error')
plt.axhline(THRESHOLD, color='r', linestyle='--', label='Threshold')
plt.legend()
plt.show()


## Interpretation
Error above threshold indicates anomaly.